In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/'
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'

In [2]:
#timeit.timeit(pd.read_csv(filepath_processed + 'Total_dataset.csv'))
start = datetime.now()
data =  pd.read_csv(filepath_processed + 'Total_dataset.csv', index_col = 0)
print ('Loading huge dataset took: ' + str((datetime.now()-start).total_seconds()) + ' seconds')

C:\Users\Dirk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dirk\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Loading huge dataset took: 43.630836 seconds


In [3]:
buurten = pd.read_csv(filepath_raw + 'postcode_huisnummer_buurten.csv',delimiter = ';')

In [4]:
buurt = pd.read_csv(filepath_raw + 'buurt2019.csv', delimiter = ';')

In [5]:
wijk = pd.read_csv(filepath_raw + 'wijk2019.csv', delimiter = ';')

In [6]:
wijk_dict = wijk.astype(str).set_index('Wijkcode2019').to_dict()['Wijknaam_2019K_NAAM']

In [7]:
buurt.Buurtcode_2019 = buurt.Buurtcode_2019.astype(str)
buurt_dict = buurt.set_index('Buurtcode_2019').to_dict()['Buurtnaam_2019']
buurten[['Buurt2019']]= buurten[['Buurt2019']].astype(str)

In [8]:
buurten[['Wijk2019']] = buurten[['Wijk2019']].astype(str)
buurten['buurt_naam'] = buurten['Buurt2019'].map(buurt_dict)
buurten['wijk_naam'] = buurten['Wijk2019'].map(wijk_dict)

In [9]:
buurten = buurten.drop(['Buurt2019','Wijk2019','Gemeente2019'], axis = 1).rename(columns = {'PC6':'postcode','Huisnummer':'huisnummer'})

In [10]:
data = data.merge(buurten, on = ['postcode','huisnummer'])

In [11]:
buurt_data_elabel = data.groupby(by = ['gemeente','buurt_naam','Energielabel']).count()[['postcode']]

In [12]:
buurten_elabels = buurt_data_elabel.pivot_table(index = ['gemeente','buurt_naam'], columns = 'Energielabel', values = 'postcode').fillna(0)

In [13]:
buurten_elabels['total'] = buurten_elabels.sum(axis = 1)

In [14]:
for i in buurten_elabels.columns[:-1]:
    buurten_elabels[i] = buurten_elabels[i] / buurten_elabels.total
buurten_elabels = buurten_elabels.reindex(buurten_elabels.columns.sort_values(ascending = False), axis=1)

In [15]:
buurt_mean_gas = data.groupby(by = ['gemeente', 'buurt_naam']).mean()[['geschat gasverbruik jaarlijks']]

In [16]:
# multiply by 8.79 to convert to kWh, divide by 1650 for connection value (vollasturen)
buurten_elabels['Mean connection value'] = buurt_mean_gas * 8.79167 / 1650

In [17]:
#Monuments
buurten_elabels['Share Monuments'] = data.groupby(by = ['gemeente','buurt_naam']).count()['CBSCATEGOR'] / buurten_elabels['total']

In [18]:
buurten_elabels.to_csv(filepath_processed + 'buurten_elabels.csv')